## Filter
One of the most powerful parameters when defining a search is the ability to filter the results.  While a Query is more of an unstructured approach to search for text within documents, a Filter provides a well-defined structured approach when we know which properties we want to restrict our matching documents.

The following examples will be using some of the popular properties as outlined within the "Properties & Metadata" notebook.  In addition, the selection of these properties was determined based on the discovery techniques used such as metadata and debugging.

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.DataPlatform.Content, 1.0.0-alpha3"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.Data.Analysis version 0.4.0

Installed package Refinitiv.DataPlatform.Content version 1.0.0-alpha3

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.DataPlatform.Content.SearchService;
using Refinitiv.DataPlatform.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Linq;
using System;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<IList<JObject>>((hits, writer) =>
{
    if (hits.Count > 0)
    {
        var fields = new List<String>();
        var rows = new List<ICollection<IHtmlContent>>();
        var data = new Dictionary<string, IHtmlContent>();

        foreach(var hit in hits)
        {
            var cells = new List<IHtmlContent>();
            foreach (var val in hit.Properties())
            {
                if ( !fields.Any(item => item.Equals(val.Name)) )
                    fields.Add(val.Name);
                data[val.Name] = td(val.Value.ToString());
            }
            rows.Add(new List<IHtmlContent>(data.Values));
            data.Keys.ToList().ForEach(x => data[x] = td(""));
        }
        
        var headers = new List<IHtmlContent>();
        headers.AddRange(fields.Select(c => (IHtmlContent)th(c)));

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [6]:
// Create a session into the desktop
var session = CoreFactory.CreateSession(new DesktopSession.Params()
                            .AppKey("Your API Key here")
                            .OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                            .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})")));
session.Open();

21/01/2021 5:36:59 PM:Session is Pending. (State: Pending)
21/01/2021 5:37:00 PM:{
  "Contents": "Desktop Session Successfully Authenticated"
}. (Event: SessionAuthenticationSuccess)
21/01/2021 5:37:00 PM:Session is Opened. (State: Opened)


#### Example - Query vs Filter
As you become more familiar with Search, you will realize there are a number of different paths you can take to arrive at the same destination.

In [7]:
var response = Search.Definition(Search.View.Instruments).Query("US046353AQ14")
                                                         .Select("DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID")
                                                         .GetData();
response.Data.Hits

"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",046353AQ1=,US046353AQ14,046353AQ1,BF4SNM7,46645702869


In [6]:
// Retrieve the document(s) for the same ISIN using a Filter
response = Search.Definition(Search.View.Instruments).Filter("ISIN eq 'US046353AQ14'")
                                                     .Select("DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID")
                                                     .GetData();
response.Data.Hits

"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",046353AQ1=,US046353AQ14,046353AQ1,BF4SNM7,46645702869


In [7]:
// If you are provided a code but are not sure if the code is an ISIN, SEDOL, CUSIP etc, 
// try using the 'CodeResolutionBase' property.
// Note: I'm using the CUSIP for the same instrument within the Filter
response = Search.Definition(Search.View.Instruments).Filter("CodeResolutionBase eq '046353AQ1'")
                                                     .Select("DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID")
                                                     .GetData();
response.Data.Hits

"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",046353AQ1=,US046353AQ14,046353AQ1,BF4SNM7,46645702869


#### Example - Apple Equities

In [8]:
// Within the view (EquityQuotes), select all documents where the term 'AAPL' is within the ticker symbol.  For each, 
// ensure the type of asset is an 'equity' and that each asset is in an active state.  I chose to filter out the 
// AssetState of 'DC' (De-activated) as my criteria.
//
// Note: An alternative to "AssetType eq 'equity'" could be to use: "RCSAssetCategoryLeaf eq 'Ordinary Share'"
var filter = "AssetType eq 'equity' and AssetState ne 'DC' and TickerSymbol eq 'AAPL'";
var select = "RIC, CompositeDescriptiveName, ExchangeCode, ExchangeName, AssetType";

response = Search.Definition(Search.View.EquityQuotes).Filter(filter)
                                                      .Select(select)
                                                      .GetData();
response.Data.Hits

AAPL.O,Apple Ord Shs APPLE ORD,NSQ,NASDAQ Global Select Consolidated,"[ ""EQUITY"" ]"
AAPLEUR.xbo,Apple Ord Shs APPLE ORD APPLE INC,XBO,MiFID Composite Cross Market Service,"[ ""EQUITY"" ]"
AAPL.OQ,Apple Ord Shs APPLE ORD,NSM,NASDAQ Stock Exchange Global Select Market,"[ ""EQUITY"" ]"
AAPL.DG,Apple Ord Shs APPLE ORD,GCD,Cboe EDGX Exchange - Nasdaq Global Select Market,"[ ""EQUITY"" ]"
AAPL.Z,Apple Ord Shs APPLE ORD APPLE INC,GCZ,Cboe BZX Exchange - Nasdaq Global Select Market,"[ ""EQUITY"" ]"
AAPL.B,Apple Ord Shs APPLE ORD,GSB,Boston SE when trading NASDAQ Global Select Market,"[ ""EQUITY"" ]"
AAPLE.MI,Apple Ord Shs APPLE ORD,MIL,Milan Stock Exchange,"[ ""EQUITY"" ]"
AAPL.ZY,Apple Ord Shs APPLE ORD APPLE INC,GCY,Cboe BYX Exchange - Nasdaq Global Select Market,"[ ""EQUITY"" ]"
AAPL.PH,Apple Ord Shs APPLE ORD APPLE INC,GSX,NASDAQ OMX PHLX Global Select Market,"[ ""EQUITY"" ]"
AAPL.DY,Apple Ord Shs APPLE ORD,GDA,Cboe EDGA Exchange - Nasdaq Global Select Market,"[ ""EQUITY"" ]"


#### Example - Vessels
List all Very Large Gas Containers (VLGC) that are in service within the last week.

In [5]:
// Build date range expression for the location of the vessel for the last 7 days
var date7DaysAgo = DateTime.Now.AddDays(-7);
var dateRange = @$"LocationTimestamp ge {date7DaysAgo:yyyy-MM-dd} and LocationTimestamp lt {DateTime.Now:yyyy-MM-dd}";
dateRange

LocationTimestamp ge 2020-12-14 and LocationTimestamp lt 2020-12-21

In [10]:
// Show the location of each vessel and some general details such as location, destination and coordinates.
filter = $"RCSAssetTypeLeaf eq 'VLGC' and VesselStatus eq 'IN SERVICE' and {dateRange}";
select = "RIC, SynonymSearchAll, DTSource, LocationTimestamp, IMO,Latitude, Longitude, LocationDestination";

response = Search.Definition(Search.View.VesselPhysicalAssets).Filter(filter)
                                                              .Select(select)
                                                              .Top(100)
                                                              .GetData();
response.Data.Hits

C}KY7309466539,Chinese Liquefied Petroleum Gas Very Large Gas Carrier,Java Sea|Singapore Tanker Zone,20/12/2020 8:10:46 AM,9002908,2.403069972992,101.733062744141,KHASAB [Khasab]
C}KY7309491970,Egyptian Liquefied Petroleum Gas Very Large Gas Carrier,Red Sea|Suez Canal Tanker Zone|Egypt,18/12/2020 5:59:44 PM,9350288,28.742584228516,33.152103424072,WADI FEIRAN [Wadi Feiran Oil Terminal]
C}KY7309491971,Egyptian Liquefied Petroleum Gas Very Large Gas Carrier,Red Sea|Suez Canal Tanker Zone|Egypt,14/12/2020 5:32:42 AM,9350290,29.597896575928,32.366004943848,SA YNB [Yanbu]
C}KY7309529540,Indonesian India Liquefied Petroleum Gas Very Large Gas Carrier,Java Sea|Lawi-Lawi Terminal Tanker Zone,20/12/2020 1:25:45 AM,9643348,-7.586550235748,114.009086608887,"KALBUT,SITUBONDO"
C}KY7309827069,Chinese Japanese American US USA Canadian Liquefied Petroleum Gas Very Large Gas Carrier Korean,East China Sea|Japan Tanker Zone,20/12/2020 11:22:32 PM,9733337,30.582277297974,130.27099609375,YEOSU S.KOREA [Yeosu (Yosu)]
C}KY7309843663,Qatari Chinese Liquefied Petroleum Gas Very Large Gas Carrier,Java Sea|Singapore Tanker Zone,18/12/2020 4:12:37 AM,9795543,1.398763298988,104.465003967285,"NINGBO, CHINA [Ningbo and Zhoushan]"
C}KY7309844136,Indonesian American US USA India American United States USA Liquefied Petroleum Gas Very Large Gas Carrier,Indonesia|East Java|Java Sea|Lawi-Lawi Terminal Tanker Zone,20/12/2020 3:08:49 AM,9752723,-7.598721504211,113.992958068848,"KALBUT,INDONESIA"
C}KY7309878957,Japanese Korean Chinese Liquefied Petroleum Gas Very Large Gas Carrier,East Sea / Sea of Japan|Korea Tanker Zone|Korea Strait Waypoint,20/12/2020 5:20:46 PM,9849289,34.82462310791,129.191375732422,PA BLB [Balboa]
C}KG7309934187,Chinese Japanese Liquefied Petroleum Gas Very Large Gas Carrier Korean,East China Sea|Japan Tanker Zone,20/12/2020 6:25:24 AM,9889552,30.588535308838,130.092483520508,YOSU [Yeosu (Yosu)]
C}KY7309466679,Chinese Liquefied Petroleum Gas Very Large Gas Carrier,Java Sea|Singapore Tanker Zone,18/12/2020 1:33:22 AM,9005053,1.357450008392,104.419586181641,OMKHS>CNDGG [Dongguan]
C}KY7309473669,Chinese Liquefied Petroleum Gas Very Large Gas Carrier,East China Sea|Shanghai Tanker Zone,18/12/2020 10:55:16 PM,9114581,25.635473251343,120.604621887207,CN JIX


#### Example - Floating Rate Notes
Retrieve all floating rate notes (floating coupon) in the healthcare sector which are issued in Euros.  For each, ensure the coupon rate is great than 2 and our Moody's rating is greater than 'Baa1'.

To determine the rating, we apply the following logic utilizing the RatingsScope property - which is a nested property providing additional details:

- RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10)

The RatingType of 'MIS' refers to the "Moody's Long-term Issuer Rating".
The CurrentRatingRank < 10 instructs search to look for ratings less than 10 ('Baa1').  **Note**: The lower the rank, the better the rating.

In [11]:
filter = @"CouponRate gt 2 and
           RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10) and
           RCSCouponTypeName eq 'Floating Coupon' and
           IsActive eq true and
           RCSTRBC2012Name eq 'Healthcare' and
           RCSCurrencyLeaf xeq 'Euro'";
select = "CouponCurrency, IssuerName, IssueDate, RIC, CouponRate, ISIN, Ticker, RCSTRBC2012Leaf, MaturityDate, AssetState";

response = Search.Definition(Search.View.GovCorpInstruments).Filter(filter)
                                                            .Select(select)
                                                            .GetData();
response.Data.Hits

EUR,Sanofi FR,01/07/1983 12:00:00 AM,FR0000140022=,15.5,FR0000140022,SASY,Pharmaceuticals (NEC)
EUR,Sanofi FR,01/10/1987 12:00:00 AM,AVEPpaEUR.MIp,15.5,FR0000047656,SASY,Pharmaceuticals (NEC)


#### Example - Outstanding Green Bonds
List the outstanding Green Bonds for Telefonica.

In [12]:
select = @"ssuerName, DocumentTitle, RIC, CouponRate, ISIN, IssuerTicker, DebtTypeDescription, 
           AmountOutstanding, FaceOutstanding";

response = Search.Definition(Search.View.GovCorpInstruments).Filter("IssuerName eq 'Telefonica' and IsGreenBond eq true")
                                                            .Select(select)
                                                            .GetData();
response.Data.Hits

"Telefonica Europe BV, Bond, TEF 2.502",ES210981985=,2.502,XS2109819859,TEFBV,Junior Subordinated Note,500000000
"Telefonica Emisiones SAU, Plain Vanilla Fixed Coupon Bond, TEF 1.069 05-Feb-2024",ES194600445=,1.069,XS1946004451,TEFEMI,Senior Note,1000000000


#### Example - Equity Index listing in the U.S.A

In [13]:
// Based on the Asset Category (Equity Index), list the top 10 indexes within the USA.
filter = "RCSIndexCountryGroupLeaf xeq 'United States' and RCSAssetCategoryLeaf xeq 'Equity Index'";

response = Search.Definition(Search.View.IndexInstruments).Filter(filter)
                                                          .Select("DocumentTitle, RIC, RCSTRBC2012Leaf")
                                                          .GetData();
response.Data.Hits

"Dow Jones Industrial Average Index, Equity Index",.DJI,Trade & Business Publishing
"S&P 500 Index - CBOE, Equity Index",.SPX,Magazine Publishing
"NASDAQ 100 Index, Equity Index",.NDX,UK Investment Trusts
"NASDAQ Composite Index, Equity Index",.IXIC,Financial & Commodity Market Operators & Service Providers (NEC)
"Refinitiv United States Price Return Index, Equity Index",.TRXFLDUSP,Professional Information Services (NEC)
"Dow Jones Transport Average Index, Equity Index",.DJT,Trade & Business Publishing
"Dow Jones Composite Index, Equity Index",.DJA,Trade & Business Publishing
"Dow Jones Utility Average Index, Equity Index",.DJU,Trade & Business Publishing
"S&P 600 Small Cap Index, Equity Index",.SPCY,Magazine Publishing
"S&P 500 Index - CME, Equity Index",.INX,Magazine Publishing


#### Example - Search for Brazil global sovereign bonds in USD with a duration greater than 1

In [14]:
filter = @"RCSIssuerCountryLeaf eq 'Brazil' and
           RCSCurrencyLeaf eq 'US Dollar' and
           RCSTRBC2012Leaf eq 'Government & Government Finance' and
           OptionAdjustedDuration gt 1";
select = "RIC, ISIN, MaturityDate, CouponRate, IndustrySector, FaceOutstandingUSD, OptionAdjustedDuration";

response = Search.Definition(Search.View.FixedIncomeInstruments).Filter(filter)
                                                                .Select(select)
                                                                .GetData();
response.Data.Hits

105756BU3=,US105756BU30,05/01/2023 12:00:00 AM,2.625,SOVERGRN,2150000000,1.975992
BR012631375=,US105756AR10,15/04/2024 12:00:00 AM,8.875,SOVERGRN,1029030000,2.939399
BR017244388=,US105756AZ36,15/04/2024 12:00:00 AM,8.875,SOVERGRN,85557000,2.939399
105756BV1=,US105756BV13,07/01/2025 12:00:00 AM,4.25,SOVERGRN,4300000000,3.682713
BR021210641=,US105756BF62,04/02/2025 12:00:00 AM,8.75,SOVERGRN,688201000,3.496739
105756CD0=,US105756CD06,06/06/2025 12:00:00 AM,2.875,SOVERGRN,1750000000,4.183824
105756BX7=,US105756BX78,07/04/2026 12:00:00 AM,6,SOVERGRN,2176245000,4.593229
105756AE0=,US105756AE07,15/05/2027 12:00:00 AM,10.125,SOVERGRN,765023000,5.040152
105756BZ2=,US105756BZ27,13/01/2028 12:00:00 AM,4.625,SOVERGRN,3000000000,5.808703
105756CA6=,US105756CA66,30/05/2029 12:00:00 AM,4.5,SOVERGRN,2000000000,6.912554


In [15]:
// Alternatively, you have the option of trying it this way which will render the same results
filter = @"IssuerCountryName eq 'Brazil' and
           OptionAdjustedDuration gt 1 and
           Currency eq 'USD' and IndustrySectorDescription eq 'Sovereign'";
select = "RIC, ISIN, MaturityDate, CouponRate, IndustrySector, FaceOutstandingUSD, OptionAdjustedDuration";

response = Search.Definition(Search.View.FixedIncomeInstruments).Filter(filter)
                                                                .Select(select)
                                                                .GetData();
response.Data.Hits

105756BU3=,US105756BU30,05/01/2023 12:00:00 AM,2.625,SOVERGRN,2150000000,1.975992
BR012631375=,US105756AR10,15/04/2024 12:00:00 AM,8.875,SOVERGRN,1029030000,2.939399
BR017244388=,US105756AZ36,15/04/2024 12:00:00 AM,8.875,SOVERGRN,85557000,2.939399
105756BV1=,US105756BV13,07/01/2025 12:00:00 AM,4.25,SOVERGRN,4300000000,3.682713
BR021210641=,US105756BF62,04/02/2025 12:00:00 AM,8.75,SOVERGRN,688201000,3.496739
105756CD0=,US105756CD06,06/06/2025 12:00:00 AM,2.875,SOVERGRN,1750000000,4.183824
105756BX7=,US105756BX78,07/04/2026 12:00:00 AM,6,SOVERGRN,2176245000,4.593229
105756AE0=,US105756AE07,15/05/2027 12:00:00 AM,10.125,SOVERGRN,765023000,5.040152
105756BZ2=,US105756BZ27,13/01/2028 12:00:00 AM,4.625,SOVERGRN,3000000000,5.808703
105756CA6=,US105756CA66,30/05/2029 12:00:00 AM,4.5,SOVERGRN,2000000000,6.912554


#### Example - Top executives

In [16]:
// Using the People View, we can utilize the OfficerDirector property, which contains a number of nested or sub-properties 
// that allow me to perform a more fine-grained search against active people that have been assigned a title code of 
// CEO (Chief Executive Officer). The results are listed based on a default rank defined by the search service.
filter = "OfficerDirector(RoleTitleCode eq 'CEO' and RoleStatus eq 'Active')";
select = @"FullName, PrimaryTitleAsReported, PrimaryOfficerDirector, RIC, 
           OfficerDirectorActiveCompanies, PrimaryOfficerDirectorRIC";
           
response = Search.Definition(Search.View.People).Filter(filter)
                                                .Select(select)
                                                .Top(20)
                                                .GetData();
response.Data.Hits

John Wendell Thompson,Independent Non-Executive Chairman of the Board,Microsoft Corp,"[ ""User Replay Ltd"", ""Crystal Decisions Inc"", ""Commerce Decisions Ltd"", ""Illumina Inc"", ""Microsoft Corp"" ]",MSFT.O
Satya Nadella,"Chief Executive Officer, Director",Microsoft Corp,"[ ""Starbucks Corp"", ""Chandler Insurance Company Ltd"", ""Microsoft Corp"" ]",MSFT.O
Art D. Levinson,Independent Chairman of the Board,Apple Inc,"[ ""Apple Inc"", ""Genentech Inc"" ]",AAPL.O
Tim D. Cook,"Chief Executive Officer, Director",Apple Inc,"[ ""Apple Inc"", ""Nike Inc"" ]",AAPL.O
Jeff P. Bezos,"Chairman of the Board, President, Chief Executive Officer",Amazon.com Inc,"[ ""Institute For Advanced Study"", ""Amazon.com Inc"" ]",AMZN.O
Keith Brian Alexander,Director,Amazon.com Inc,"[ ""Amazon.com Inc"", ""Ironnet Cybersecurity Inc"" ]",AMZN.O
Jon J. Rubinstein,Independent Director,Amazon.com Inc,"[ ""Palm Inc"", ""Amazon.com Inc"" ]",AMZN.O
Sundar Pichai,"Chief Executive Officer, Director",Alphabet Inc,"[ ""Alphabet Inc"", ""Ruba Inc (Fka Kudo Inc)"" ]",GOOGL.O
Roger W. Ferguson,Independent Director,Alphabet Inc,"[ ""Teachers Insurance and Annuity Association of America"", ""Audax Health Solutions LLC"", ""Alphabet Inc"", ""International Flavors & Fragrances Inc"", ""General Mills Inc"", ""Memorial Sloan-Kettering Cancer Center"" ]",GOOGL.O
Mark Zuckerberg,"Chairman of the Board, Chief Executive Officer, Founder",Facebook Inc,"[ ""Facebook Inc"" ]",FB.O
Dustin A. Moskovitz,Venture Economics Executive,Facebook Inc,"[ ""Facebook Inc"", ""Asana Inc"" ]",FB.O


#### Example - Live Options
The following example will specify an underlying instrument (RIC) and pull the list of options that are active, i.e. not expired.

In [9]:
// Define our underlying
var underlying = "AAPL.O";

// To search for our instrument, I'm utilizing the property: 'UnderlyingQuoteRIC'.  The property does not support an 
// exact match 'xeq', so we must use the partial match 'eq' to find all hits. Depending on the instrument you specify, 
// this may introduce additional hits.  For example, if you are using the underlying 'AAPL.O', search will also find a 
// hit on 'AAPL.OQ', for example. To protect against this, we can utilize a combination of the built-in functions, 
// 'startswith' and 'endwith' to perform our match.
filter = @$"AssetState eq 'AC' and
            startswith(UnderlyingQuoteRIC,'{underlying}') and
            endswith(UnderlyingQuoteRIC,'{underlying}') and
            RCSAssetClass eq 'OPT' and IsChain eq false and
            ExpiryDate ge {DateTime.Now:yyyy-MM-dd}";
            
select = "DocumentTitle, RIC, ExpiryDate, CallPutOption, StrikePrice";
           
response = Search.Definition(Search.View.EquityDerivativeQuotes).Filter(filter)
                                                                .Select(select)
                                                                .Top(10000)
                                                                .GetData();
response.Data.Hits

"OPRA Apple Equity Option 120 Put Feb 2021 , Equity Cash Option, Put 120 USD 19-Feb-2021, OPRA",AAPLN192112000.U,19/02/2021 12:00:00 AM,Put,120
"OPRA Apple Equity Option 130 Call Feb 2021 , Equity Cash Option, Call 130 USD 19-Feb-2021, OPRA",AAPLB192113000.U,19/02/2021 12:00:00 AM,Call,130
"OPRA Apple Equity Option 125 Call Feb 2021 , Equity Cash Option, Call 125 USD 19-Feb-2021, OPRA",AAPLB192112500.U,19/02/2021 12:00:00 AM,Call,125
"OPRA Apple Equity Option 130 Put Feb 2021 , Equity Cash Option, Put 130 USD 19-Feb-2021, OPRA",AAPLN192113000.U,19/02/2021 12:00:00 AM,Put,130
"OPRA Apple Equity Option 120 Call Feb 2021 , Equity Cash Option, Call 120 USD 19-Feb-2021, OPRA",AAPLB192112000.U,19/02/2021 12:00:00 AM,Call,120
"OPRA Apple Equity Option 140 Call Feb 2021 , Equity Cash Option, Call 140 USD 19-Feb-2021, OPRA",AAPLB192114000.U,19/02/2021 12:00:00 AM,Call,140
"OPRA Apple Equity Option 125 Put Feb 2021 , Equity Cash Option, Put 125 USD 19-Feb-2021, OPRA",AAPLN192112500.U,19/02/2021 12:00:00 AM,Put,125
"OPRA Apple Equity Option 135 Call Feb 2021 , Equity Cash Option, Call 135 USD 19-Feb-2021, OPRA",AAPLB192113500.U,19/02/2021 12:00:00 AM,Call,135
"OPRA Apple Equity Option 135 Put Feb 2021 , Equity Cash Option, Put 135 USD 19-Feb-2021, OPRA",AAPLN192113500.U,19/02/2021 12:00:00 AM,Put,135
"OPRA Apple Equity Option 145 Call Feb 2021 , Equity Cash Option, Call 145 USD 19-Feb-2021, OPRA",AAPLB192114500.U,19/02/2021 12:00:00 AM,Call,145
"OPRA Apple Equity Option 140 Put Feb 2021 , Equity Cash Option, Put 140 USD 19-Feb-2021, OPRA",AAPLN192114000.U,19/02/2021 12:00:00 AM,Put,140
